## Churn prediction
#### Author: Aleksandra Kocot

Let's try to predict users churn based on dataset from Kaggle competition:
<br>
><b>"WSDM - KKBox's Churn Prediction Challenge
Can you predict when subscribers will churn?"</b>
<br>
https://www.kaggle.com/c/kkbox-churn-prediction-challenge/

<hr>
<h3> Introduction </h3>
The churning user is the one that does not renew the subscription within the 30 days after the current subscription expiration.

**Churn definition (in data description on Kaggle)** 
>The criteria of "churn" is no new valid service subscription within 30 days after the current membership expires.

In [99]:
import xgboost as xgb

### Non-contractual vs conctractual (subscription based) business model

In [12]:
import pandas as pd
from os.path import join

### Loading the data

The files set provided contains bigger files with data till the end of February 2017 and additional files ("v2" suffix) with update of March 2017.
They are too big to load them all into python but we can concatenate in terminal.
In Linux terminal:
<br><code>cat transactions.csv > transactions_v3.csv; tail -n +2 transactions_v2.csv >> transactions_v3.csv </code>

In [17]:
PROJECT_DIR = r"C:\Users\Olks\Desktop\churn_prediction"

In [43]:
train_path = join(PROJECT_DIR, "train.csv")
train_v2_path = join(PROJECT_DIR, "train_v2.csv")
user_logs_path = join(PROJECT_DIR, "user_logs.csv")
user_logs_v2_path = join(PROJECT_DIR, "user_logs_v2.csv")
sample_submission_zero_path = join(PROJECT_DIR, "sample_submission_zero.csv")
sample_submission_v2_path = join(PROJECT_DIR, "sample_submission_v2.csv")
transactions_path = join(PROJECT_DIR, "transactions.csv")
transactions_v2_path = join(PROJECT_DIR, "transactions_v2.csv")
transactions_v3_path = join(PROJECT_DIR, "transactions_v3.csv")
members_v3_path = join(PROJECT_DIR, "members_v3.csv")

In [ ]:
# nrows=100
# usecols=["msno",...]

train = pd.read_csv(train_path, nrows=100)
train_v2 = pd.read_csv(train_v2_path)
user_logs = pd.read_csv(user_logs_path, usecols=["msno", "date"]) 
user_logs_v2 = pd.read_csv(user_logs_v2_path, usecols=["msno", "date"])
sample_submission_zero = pd.read_csv(sample_submission_zero_path, nrows=100)
sample_submission_v2 = pd.read_csv(sample_submission_v2_path, nrows=100)
transactions = pd.read_csv(transactions_path, nrows=100)
transactions_v2 = pd.read_csv(transactions_v2_path, nrows=100)
transactions_v3 = pd.read_csv(transactions_v3_path, , usecols=["msno", "transaction_date", "is_cancel"])
members_v3 = pd.read_csv(members_v3_path, nrows=100)

### Exploration of the data

In [128]:
# TODO: !

### First submission

In [48]:
sample_submission_v2 = pd.read_csv(sample_submission_v2_path)

In [69]:
train_v2 = pd.read_csv(train_v2_path)

In [66]:
transactions_v3 = pd.read_csv(transactions_v3_path, usecols=["msno", "transaction_date", "membership_expire_date", "is_cancel"])

In [70]:
transactions = pd.read_csv(transactions_path, usecols=["msno", "transaction_date", "membership_expire_date", "is_cancel"])

In [71]:
transactions_v2 = pd.read_csv(transactions_v2_path, usecols=["msno", "transaction_date", "membership_expire_date", "is_cancel"])

In [85]:
train = pd.read_csv(train_path)

In [67]:
transactions_v3.dtypes

msno                      object
transaction_date           int64
membership_expire_date     int64
is_cancel                  int64
dtype: object

In [68]:
transactions_v3

,msno,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,20150930,20151121,0
...,...,...,...,...
22978750,zwF50wwaJI2TBKWhB42HRBJ6EQK0jgSo1Xmwb9Jq3SU=,20170215,20170817,0
22978751,zx/h5MzQQmsSat04wSfGpHp6N8aWLLwM1+7OV7ujmPY=,20170306,20170406,0
22978752,zxvgjIKjy18Fm+cIWUfYKr68z09+ILBxuMW0DnbeUZ8=,20170308,20170408,0
22978753,zzNhkExbpzmpjp9tXefiCUBtgNLgS+vZE7fFfTRDJVc=,20170318,20170417,0


### Features

1. **days_from_start** = Number of days from first transaction
2. **transactions_num** = Number or transactions
3. **calnceletions_num** = Number of cancelations

In [105]:
def calculate_transactional_features(transactions_table, cutoff_day):

    users_transactions_features = transactions_table.groupby("msno").agg({"transaction_date": ["min", "count"], 
                                                                           "is_cancel": "sum"})
    
    users_transactions_features.columns = [a + "_" + b for a,b in users_transactions_features.columns]
    
    users_transactions_features["first_transaction"] = pd.to_datetime(users_transactions_features.transaction_date_min, 
                                                                  format='%Y%m%d')
    
    train_cutoff_day = pd.to_datetime(cutoff_day, format='%Y%m%d') 
    
    users_transactions_features["days_from_start"] = (train_cutoff_day - users_transactions_features["first_transaction"]).dt.days
    
    return users_transactions_features.reset_index()

In [89]:
train_features = calculate_transactional_features(transactions, '20170301')

In [106]:
test_features = calculate_transactional_features(transactions_v3, '20170401')

In [107]:
test_features

,msno,transaction_date_min,transaction_date_count,is_cancel_sum,first_transaction,days_from_start
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,20160909,1,0,2016-09-09,204
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20151121,2,0,2015-11-21,497
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,20161116,5,0,2016-11-16,136
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20150131,21,0,2015-01-31,791
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,20150126,27,0,2015-01-26,796
...,...,...,...,...,...,...
2426138,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,20151014,14,1,2015-10-14,535
2426139,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,20170205,2,0,2017-02-05,55
2426140,zzzN9thH22os1dRS0VHReY/8FTfGHOi86//d+wGGFsQ=,20160304,12,1,2016-03-04,393
2426141,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,20150608,1,0,2015-06-08,663


In [108]:
sample_submission_v2

,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0
...,...,...
907466,8nS1SbsI9+9vN07m8GyfmsWv/A9H8dUlbIbjbZ2a/Nk=,0
907467,KsOpJUA4P2pEaSl0afZhyl1QBaLHXTw33IYuQsnKdxE=,0
907468,NIcI2YUj9v2tC/NRtreLz1wxwLczQacJ2Ek9RJKgarI=,0
907469,b8MlVUtzUhQrIqKcqVKPENOIUo/5EiZxfExBkkfoBxY=,0


In [109]:
test_with_features = sample_submission_v2.drop(columns="is_churn").merge(test_features)

In [110]:
test_with_features

,msno,transaction_date_min,transaction_date_count,is_cancel_sum,first_transaction,days_from_start
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,20150718,21,0,2015-07-18,623
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,20150331,23,0,2015-03-31,732
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,20150131,11,2,2015-01-31,791
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,20151127,17,0,2015-11-27,491
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,20161222,4,0,2016-12-22,100
...,...,...,...,...,...,...
907466,8nS1SbsI9+9vN07m8GyfmsWv/A9H8dUlbIbjbZ2a/Nk=,20160307,13,0,2016-03-07,390
907467,KsOpJUA4P2pEaSl0afZhyl1QBaLHXTw33IYuQsnKdxE=,20150107,29,1,2015-01-07,815
907468,NIcI2YUj9v2tC/NRtreLz1wxwLczQacJ2Ek9RJKgarI=,20160718,9,0,2016-07-18,257
907469,b8MlVUtzUhQrIqKcqVKPENOIUo/5EiZxfExBkkfoBxY=,20150116,27,0,2015-01-16,806


In [90]:
train_features

,msno,transaction_date_min,transaction_date_count,is_cancel_sum,first_transaction,days_from_start
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,20160909,1,0,2016-09-09,173
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20151121,1,0,2015-11-21,466
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,20161116,4,0,2016-11-16,105
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20150131,19,0,2015-01-31,760
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,20150126,26,0,2015-01-26,765
...,...,...,...,...,...,...
2363621,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,20151014,14,1,2015-10-14,504
2363622,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,20170205,1,0,2017-02-05,24
2363623,zzzN9thH22os1dRS0VHReY/8FTfGHOi86//d+wGGFsQ=,20160304,12,1,2016-03-04,362
2363624,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,20150608,1,0,2015-06-08,632


In [91]:
train_features.dtypes

msno                              object
transaction_date_min               int64
transaction_date_count             int64
is_cancel_sum                      int64
first_transaction         datetime64[ns]
days_from_start                    int64
dtype: object

In [92]:
train_with_features = train.merge(train_features)

In [94]:
train

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1
...,...,...
992926,tUM0yxAj50Vc35vQZ++xMlomdyeLJUW9lEcoPnt3H+g=,0
992927,KQS8etmfGgvE/7Y9gK+E9wdLnRTI0lyyPXaXL3I8E4c=,0
992928,8/jDLgNREuWI9hcKVYp8723nmavn01T+AuMWkK3uM7g=,0
992929,iNV99F1Rml7EMndOeLI0Y/iek6aCj/Qp1Z4dZvR+sak=,0


In [93]:
train_with_features

,msno,is_churn,transaction_date_min,transaction_date_count,is_cancel_sum,first_transaction,days_from_start
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,20161031,2,0,2016-10-31,121
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,20150131,23,2,2015-01-31,760
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,20160202,10,1,2016-02-02,393
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,20160104,2,0,2016-01-04,422
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,20150102,8,0,2015-01-02,789
...,...,...,...,...,...,...,...
992926,tUM0yxAj50Vc35vQZ++xMlomdyeLJUW9lEcoPnt3H+g=,0,20170201,2,0,2017-02-01,28
992927,KQS8etmfGgvE/7Y9gK+E9wdLnRTI0lyyPXaXL3I8E4c=,0,20170219,1,0,2017-02-19,10
992928,8/jDLgNREuWI9hcKVYp8723nmavn01T+AuMWkK3uM7g=,0,20170211,2,1,2017-02-11,18
992929,iNV99F1Rml7EMndOeLI0Y/iek6aCj/Qp1Z4dZvR+sak=,0,20170215,3,1,2017-02-15,14


In [ ]:
user_logs_v2.date.describe()

In [35]:
print(f"Table size: {members_v3.memory_usage().sum() / 2**10} KB")

Table size: 4.8125 KB


In [36]:
user_logs.dtypes

msno           object
date            int64
num_25          int64
num_50          int64
num_75          int64
num_985         int64
num_100         int64
num_unq         int64
total_secs    float64
dtype: object

 Let's base our solution on tips that the challenge winner, Bryian Gregory, gives in his article, <br>
**"Predicting Customer Churn: Extreme Gradient Boosting with Temporal Data"**
<br>
https://medium.com/@bryan.gregory1/predicting-customer-churn-extreme-gradient-boosting-with-temporal-data-332c0d9f32bf

The data is too big to read all at once. However, to train a model we need user level data.
Therefore, we can read the data by chunks and update our user table.

Features: 
    1. Days passed from last day when user played at least 85% of the song

In [ ]:
file_path = None

for pdf in pd.read_csv(file_pat, chunksize=1000):
    *do something here*

### Modeling with XGboost
https://www.datacamp.com/community/tutorials/xgboost-in-python?utm_source=adwords_ppc&utm_campaignid=9942305733&utm_adgroupid=100189364546&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034352&utm_targetid=aud-299261629574:dsa-929501846124&utm_loc_interest_ms=&utm_loc_physical_ms=9060640&gclid=Cj0KCQjw0YD4BRD2ARIsAHwmKVkoOKq4tg0ihd-a5jx70ZIDySVNt1e8bTrBYOOQkPIaLb5aElQY3FgaApotEALw_wcB

In [96]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(train_with_features, test_size=0.2)

In [97]:
train_set

,msno,is_churn,transaction_date_min,transaction_date_count,is_cancel_sum,first_transaction,days_from_start
224572,+vRaxpqi2VyVQgo98lEw7twTUj8oxO+UHVm7x6dccRU=,0,20151121,16,0,2015-11-21,466
724504,HtVKhrs6hOZZfE3eI0GLbXtKFy+6GWtu7jo7s6CFDsI=,0,20150105,26,0,2015-01-05,786
526559,ma0baoBEwInk1HTKvV+jyg5gqAEB5M9A/olJU5NNGw8=,0,20160116,14,0,2016-01-16,410
489355,XEmwQoo2+30UJ2DT17BArguLI55Ig5Q54T9cPEKVlzI=,0,20160206,13,0,2016-02-06,389
110565,+cx9DilnBKIBsFKlvTClH6QkhtBV/1DuEENamtWkcVE=,0,20150124,26,0,2015-01-24,767
...,...,...,...,...,...,...,...
60967,OhF4HCKZDnAbXszhdb0iO7umLtyfKEx8H3MHm48YYEQ=,1,20150114,26,1,2015-01-14,777
373552,VhFidAZHrjzczHquUJHCAxBCU8H6mGzXEZ6P9/atoto=,0,20160205,13,0,2016-02-05,390
477895,N0joBhx9zFwAzWTz+Cw/6mIjZLYL3xrkRmWm9Xbicjw=,0,20150125,26,1,2015-01-25,766
592292,A+wMnn0mbCWGOVoPR+rlqcRIfRO/VSGqWCIINWI07Lw=,0,20160319,12,0,2016-03-19,347


In [103]:
#XGBoost Model
features = ["transaction_date_count", "is_cancel_sum", "days_from_start"]
X_train, y_train = train_set[features], train_set["is_churn"]
X_test, y_test = test_set[features], test_set["is_churn"]

xgb_model = xgb.XGBClassifier(max_depth=7, 
                              learning_rate=0.1,
                              objective= 'binary:logistic',
                              n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = xgb_model.predict(X_test)


Accuracy of XGB classifier on training set: 0.95
Accuracy of XGB classifier on test set: 0.95


In [111]:
submission = xgb_model.predict_proba(test_with_features[features])

In [113]:
submission_int = xgb_model.predict(test_with_features[features])

In [119]:
pd.DataFrame(submission)

,0,1
0,0.984811,0.015189
1,0.964037,0.035963
2,0.691230,0.308770
3,0.997154,0.002846
4,0.982727,0.017273
...,...,...
907466,0.994726,0.005274
907467,0.996033,0.003967
907468,0.990980,0.009020
907469,0.988826,0.011174


In [124]:
final_submission = pd.DataFrame({"msno": test_with_features.msno , "is_churn": pd.DataFrame(submission).loc[:,1]})

### Log loss calculated by Kaggle -> 0.15289   (336 place on Public Leaderboard/ 576)

In [125]:
final_submission

,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0.015189
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0.035963
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0.308770
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0.002846
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0.017273
...,...,...
907466,8nS1SbsI9+9vN07m8GyfmsWv/A9H8dUlbIbjbZ2a/Nk=,0.005274
907467,KsOpJUA4P2pEaSl0afZhyl1QBaLHXTw33IYuQsnKdxE=,0.003967
907468,NIcI2YUj9v2tC/NRtreLz1wxwLczQacJ2Ek9RJKgarI=,0.009020
907469,b8MlVUtzUhQrIqKcqVKPENOIUo/5EiZxfExBkkfoBxY=,0.011174


In [127]:
final_submission.to_csv("submission_2.csv", index=False)

In [ ]:
# TODO: try it out!
data_dmatrix = xgb.DMatrix(data=X,label=y)
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)